In [180]:
import pandas as pd
import numpy as np

In [153]:
df_train = pd.read_csv('../input/train_sample.csv')

In [7]:
df_train.shape

(100000, 8)

In [19]:
cols = ['ip', 'day', 'hour']

In [22]:
df_train.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'click_time', 'attributed_time',
       'is_attributed'],
      dtype='object')

In [23]:
df = df_train

In [27]:
df['click_time'] = pd.to_datetime(df['click_time'])

In [28]:
dt = df['click_time'].dt

In [33]:
df['day'] = dt.day.astype('uint8')
df['hour'] = dt.hour.astype('uint8')
del(dt)

In [40]:
arr_slice = df[cols].values

In [53]:
arr_slice.max(0)

array([364757,      9,     23])

In [52]:
arr_slice

array([[ 87540,      7,      9],
       [105560,      7,     13],
       [101424,      7,     18],
       ...,
       [ 18839,      8,     11],
       [114276,      8,     17],
       [119349,      7,     14]])

In [54]:
arr_slice.shape

(100000, 3)

In [ ]:
np.uni

In [56]:
unq, unqtags, counts = np.unique(np.ravel_multi_index(arr_slice.T, arr_slice.max(0)+1), return_inverse=True, return_counts=True)

In [58]:
unq.shape

(89853,)

In [63]:
nrmi = np.ravel_multi_index(arr_slice.T, arr_slice.max(0)+1)

In [69]:
np.unique(nrmi).shape

(89853,)

In [74]:
a = [1,1,1,1, 3,3,2]

In [75]:
np.unique(a, return_inverse=True, return_counts=True)

(array([1, 2, 3]), array([0, 0, 0, 0, 2, 2, 1]), array([4, 1, 2]))

In [82]:
def df_add_counts(df, cols):
	arr_slice = df[cols].values
	unq, unqtags, counts = np.unique(np.ravel_multi_index(arr_slice.T, arr_slice.max(0) + 1),
									 return_inverse=True, return_counts=True)
	df["_".join(cols)+'_count'] = counts[unqtags]

In [ ]:
def df2csr(wb, df, pick_hours=None):
	df.reset_index(drop=True, inplace=True)
	with timer("Adding counts"):
		df['click_time']= pd.to_datetime(df['click_time'])
		dt= df['click_time'].dt
		df['day'] = dt.day.astype('uint8')
		df['hour'] = dt.hour.astype('uint8')
		del(dt)
		df_add_counts(df, ['ip', 'day', 'hour'])
		df_add_counts(df, ['ip', 'app'])
		df_add_counts(df, ['ip', 'app', 'os'])
		df_add_counts(df, ['ip', 'device'])
		df_add_counts(df, ['app', 'channel'])
		#cpuStats()

In [83]:
df_add_counts(df, ['ip', 'day', 'hour'])

In [85]:
df_add_counts(df, ['ip', 'app'])
df_add_counts(df, ['ip', 'app', 'os'])
df_add_counts(df, ['ip', 'device'])
df_add_counts(df, ['app', 'channel'])


In [ ]:
df['category'] = (df['ip'].astype(str) + "_" + df['app'].astype(str) + "_" + df['device'].astype(str) \
                 + "_" + df['os'].astype(str) + "_" + df['channel'].astype(str)).apply(hash) % D

In [105]:
click_buffer= np.full(D, 3000000000, dtype=np.uint32)

In [108]:
D

67108864

In [107]:
click_buffer.shape

(67108864,)

In [109]:
df['click_time'][0]

Timestamp('2017-11-07 09:30:38')

In [110]:
df['epochtime']= df['click_time'].astype(np.int64) // 10 ** 9

In [129]:
next_clicks= []
for category, time in zip(reversed(df['category'].values), reversed(df['epochtime'].values)):
    next_clicks.append(click_buffer[category]-time)
    click_buffer[category]= time
    

In [136]:
del(click_buffer)
df['next_click']= list(reversed(next_clicks))

In [101]:
D = 2**26

In [87]:
df.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'click_time', 'attributed_time',
       'is_attributed', 'day', 'hour', 'ip_day_hour_count', 'ip_app_count',
       'ip_app_os_count', 'ip_device_count', 'app_channel_count'],
      dtype='object')

In [140]:
for fea in ['ip_day_hour_count','ip_app_count','ip_app_os_count','ip_device_count',
            'app_channel_count','next_click']:  
    df[fea]= np.log2(1 + df[fea].values).astype(int)

In [143]:
str_array= ("I" + df['ip'].astype(str) \
			+ " A" + df['app'].astype(str) \
			+ " D" + df['device'].astype(str) \
			+ " O" + df['os'].astype(str) \
			+ " C" + df['channel'].astype(str) \
			+ " WD" + df['day'].astype(str) \
			+ " H" + df['hour'].astype(str) \
			+ " AXC" + df['app'].astype(str)+"_"+df['channel'].astype(str) \
			+ " OXC" + df['os'].astype(str)+"_"+df['channel'].astype(str) \
			+ " AXD" + df['app'].astype(str)+"_"+df['device'].astype(str) \
			+ " AXOCXC" + df['app'].astype(str)+"_"+df['os'].astype(str) \
			  +"_"+df['channel'].astype(str) \
			+ " IXA" + df['ip'].astype(str)+"_"+df['app'].astype(str) \
			+ " AXO" + df['app'].astype(str)+"_"+df['os'].astype(str) \
			+ " IDHC" + df['ip_day_hour_count'].astype(str) \
			+ " IAC" + df['ip_app_count'].astype(str) \
			+ " AOC" + df['ip_app_os_count'].astype(str) \
			+ " IDC" + df['ip_device_count'].astype(str) \
			+ " AC" + df['app_channel_count'].astype(str) \
			+ " NC" + df['next_click'].astype(str)
		  ).values

In [160]:
df['is_attributed'].values

array([0, 0, 0, ..., 0, 0, 0])

In [184]:
batchsize = 10000000

In [182]:
from wordbatch.extractors import WordHash

In [185]:
wb = wordbatch.WordBatch(None, extractor=(WordHash, {"ngram_range": (1, 1), "analyzer": "word",
													 "lowercase": False, "n_features": D,
													 "norm": None, "binary": True})
						 , minibatch_size=batchsize // 80, procs=8, freeze=True, timeout=1800, verbose=0)


In [186]:
wb

In [188]:
str_array[0]

'I87540 A12 D1 O13 C497 WD7 H9 AXC12_497 OXC13_497 AXD12_1 AXOCXC12_13_497 IXA87540_12 AXO12_13 IDHC1 IAC2 AOC1 IDC3 AC7 NC30'

In [189]:
wb.transform(str_array)

<100000x67108864 sparse matrix of type '<class 'numpy.float64'>'
	with 1901082 stored elements in Compressed Sparse Row format>

In [190]:
X= wb.transform(str_array)

In [191]:
str_array.shape

(100000,)

In [194]:
X[0]

<1x67108864 sparse matrix of type '<class 'numpy.float64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [213]:
X.shape

(100000, 67108864)

In [212]:
X.getrow(1).toarray().shape

(1, 67108864)

In [161]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import *
vct = HashingVectorizer()
clf = SGDRegressor()

/Users/a407688/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [162]:
import wordbatch

In [163]:
from wordbatch.extractors import WordBag
from wordbatch.models import FTRL

/Users/a407688/anaconda/lib/python3.6/site-packages/randomstate/__init__.py:66: RandomStateDeprecationWarning: 
**End-of-life notification**

This library was designed to bring alternative generators to the NumPy 
infrastructure. It as been successful in advancing the conversation 
for a future implementation of a new random number API in NumPy which 
will allow new algorithms and/or generators. The next step
in this process is to separate the basic (or core RNG) from the 
functions that transform random bits into useful random numbers.
This has been implemented in a successor project  **randomgen** 
available on GitHub

https://github.com/bashtage/randomgen

or PyPi

https://pypi.org/project/randomstate/.

randomgen has a slightly different API, so please see the randomgen documentation

https://bashtage.github.io/randomgen.

  warnings.warn(DEPRECATION_MESSAGE, RandomStateDeprecationWarning)


In [164]:
wb = wordbatch.WordBatch(extractor=(WordBag, {
    "hash_ngrams":2,
    'hash_ngrams_weights':[0.5, -1.0],
    "hash_size":2**23,
    "norm":'l2',
    "tf":'log',
    'idf':50.0
}))

In [165]:
wb

In [166]:
clf = FTRL(alpha=1.0, beta=1.0, L1=0.00001, L2=1.0, D=2**25, iters=1)

In [167]:
train_texts= ["Cut down a tree with a herring? It can't be done.", "Don't say that word.", "How can we not say the word if you don't tell us what it is?"]

In [168]:
train_labels=[1,0,1]

In [169]:
test_texts= ["Wait! I said it! I said it! Ooh! I said it again!"]



In [171]:
a = wb.transform(train_texts)

Normalize text
Extract wordbags


In [179]:
clf.fit(wb.transform(train_texts), train_labels)
preds= clf.predict(wb.transform(test_texts))


Normalize text
Extract wordbags
Total e: 1.5816271119805094
Normalize text
Extract wordbags
